In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision import models

In [2]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize for VGG
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
])

# Load CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# Load pre-trained VGG16 model
model = models.vgg16(pretrained=True)

# Modify the classifier to match CIFAR-10 classes
model.classifier[6] = nn.Linear(model.classifier[6].in_features, 10)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [5]:

      running_loss = 0.0
      for i, data in enumerate(train_loader, 0):
          inputs, labels = data  # Get the inputs and labels

          # Move inputs and labels to the same device as the model
          inputs, labels = inputs.to(device), labels.to(device)

          optimizer.zero_grad()  # Zero the parameter gradients
          outputs = model(inputs)  # Forward pass
          loss = criterion(outputs, labels)  # Compute loss
          loss.backward()  # Backward pass
          optimizer.step()  # Optimize the weights

          running_loss += loss.item()
          if i % 100 == 99:  # Print every 100 mini-batches
              print(f' Step [{i + 1}/{len(train_loader)}], Loss: {running_loss / 100:.4f}')
              running_loss = 0.0

print('Finished Training')

 Step [100/782], Loss: 2.2885
 Step [200/782], Loss: 2.2748
 Step [300/782], Loss: 1.9804
 Step [400/782], Loss: 1.6585
 Step [500/782], Loss: 1.3654
 Step [600/782], Loss: 1.0843
 Step [700/782], Loss: 0.8683
Finished Training


In [6]:
# Function to evaluate the model
def evaluate_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Evaluate the model on the test set
accuracy = evaluate_model(model, test_loader)
print(f'Accuracy of the model on the CIFAR-10 test set: {accuracy:.2f}%')

Accuracy of the model on the CIFAR-10 test set: 78.53%


In [7]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import VGG16, VGG19, InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize pixel values
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Normalize pixel values to [0, 1]
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [11]:
# Create the training and validation generators
train_generator = datagen.flow(x_train, y_train, batch_size=32)  # Adjust batch size as needed
validation_generator = datagen.flow(x_test, y_test, batch_size=32)

# Initialize the VGG16 model
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))


# Train VGG19

vgg19_model = VGG19(weights='imagenet', include_top=False, input_shape=(32, 32, 3))


In [12]:
def create_model(base_model):
    model = models.Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(10, activation='softmax'))  # 10 classes for CIFAR-10
    return model

# Create models
vgg16_final_model = create_model(vgg16_model)
vgg19_final_model = create_model(vgg19_model)

In [13]:
# Compile models
vgg16_final_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
vgg19_final_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [14]:
# Train VGG16
vgg16_history = vgg16_final_model.fit(datagen.flow(x_train, y_train, batch_size=64),
                                       epochs=10,
                                       validation_data=(x_test, y_test))

# Train VGG19
vgg19_history = vgg19_final_model.fit(datagen.flow(x_train, y_train, batch_size=64),
                                       epochs=10,
                                       validation_data=(x_test, y_test))




Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


782/782 ━━━━━━━━━━━━━━━━━━━━ 69s 74ms/step - accuracy: 0.1009 - loss: 2.3306 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 65s 63ms/step - accuracy: 0.1013 - loss: 2.3027 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 82s 63ms/step - accuracy: 0.0984 - loss: 2.3027 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 51s 64ms/step - accuracy: 0.0990 - loss: 2.3027 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 80s 62ms/step - accuracy: 0.0978 - loss: 2.3027 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 91s 74ms/step - accuracy: 0.0943 - loss: 2.3027 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 81s 73ms/step - accuracy: 0.0964 - loss: 2.3027 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 73s 62ms/step - accuracy: 0.0969 - loss: 2.3027 - val_accurac

In [15]:
# Get validation accuracies
vgg16_val_acc = max(vgg16_history.history['val_accuracy'])
vgg19_val_acc = max(vgg19_history.history['val_accuracy'])

print(f'VGG16 Validation Accuracy: {vgg16_val_acc:.2f}')
print(f'VGG19 Validation Accuracy: {vgg19_val_acc:.2f}')

VGG16 Validation Accuracy: 0.10
VGG19 Validation Accuracy: 0.10
